# Extending EAGO for a Custom Lower-Bounding Problem: An $\alpha$ BB example for a QCQP.
In this example, we will demonstrate the use of a user-defined lower-bounding problem that uses $\alpha$ BB convex relaxations. In this example, we wish to solve the (nonconvex) QCQP:
$$\begin{align*}
&\min_{\mathbf x\in X\in \mathbb{IR}^2}\frac{1}{2}\mathbf x^{\rm T}\mathbf Q_f\mathbf x+\mathbf c_f^{\rm T}\mathbf x\\
{\rm s.t.}\;\;&g_1(\mathbf x)=\frac{1}{2}\mathbf x^{\rm T}\mathbf Q_{g_1}\mathbf x+\mathbf c_{g_1}^{\rm T}\mathbf x\le 0\\
&g_2(\mathbf x)=\frac{1}{2}\mathbf x^{\rm T}\mathbf Q_{g_2}\mathbf x+\mathbf c_{g_2}^{\rm T}\mathbf x\le 0
\end{align*}
$$
with $\mathbf Q_i\in\mathbb R^{2\times 2}$ not positive semidefinite for any $i$.

We start by loading the necessary packages. This notebook was tested working using Julia 1.9.1 with JuMP v1.12.0, EAGO v0.8.1, and Ipopt v1.4.1.

In [1]:
using JuMP, EAGO, Ipopt, LinearAlgebra

For convenience, we'll define the following function that returns all the problem data $\mathbf Q_i$ and $\mathbf c_i$.

In [2]:
function QCQP_setup()

    Qf = [3. 3/2.;3/2. -5.]
    cf = [3.;2.]

    Qg1 = [-2. 5.;5. -2.]
    cg1 = [1.;3.]

    Qg2 = [-6. 3.;3. 2.]
    cg2 = [2.;1.]
    
    return Qf,cf,Qg1,cg1,Qg2,cg2
end

QCQP_setup (generic function with 1 method)

The next function we'll define will take as input data for a particular quadratic function and the interval bounds on the decision variables, and construct an $\alpha$ BB convex relaxation of that function. Since we're solving a QCQP, we'll use the $\verb|eigvals|$ function to directly compute the eigenvalues of the input $\mathbf Q_i$ matrix.

In [3]:
function αBB_relax(Q::Matrix{T},c::Vector{T},xL::Vector{T},xU::Vector{T},x::Real...) where {T<:Float64}
    α=max(0,-minimum(eigvals(Q))/2)
    y = [x[1];x[2]]
    cv = 1/2*y'*Q*y+c'*y+α*(xL-y)'*(xU-y)
    return cv
end

αBB_relax (generic function with 1 method)

The following code first defines our EAGO extension (custom version) struct and then it redefines the lower-bounding problem as our own version. That is, when we call this customized version of EAGO to solve the problem, it'll deploy this version of the lower-bounding problem instead of the default version.  

In [4]:
import EAGO: Optimizer, GlobalOptimizer

struct αBB_Convex <: EAGO.ExtensionType end
import EAGO: lower_problem!
function EAGO.lower_problem!(t::αBB_Convex, opt::GlobalOptimizer)
    # Get active node
    n = opt._current_node
    # Get bounds on active node for calculating relaxations
    xL = n.lower_variable_bounds[1:2]
    xU = n.upper_variable_bounds[1:2]
    # Get the problem data
    Qf,cf,Qg1,cg1,Qg2,cg2=QCQP_setup()

    # Define the JuMP model and declare the solver, define the variables
    mL = JuMP.Model(JuMP.optimizer_with_attributes(Ipopt.Optimizer,
                                "print_level" => 0))
    @variable(mL,xL[i]<=x[i=1:2]<=xU[i])
    
    # Define the function closures for the user-defined relaxations
    fcv(x...)  = αBB_relax(Qf,cf,xL,xU,x...)   # relaxation of objective
    g1cv(x...) = αBB_relax(Qg1,cg1,xL,xU,x...) # relaxation of constraint 1
    g2cv(x...) = αBB_relax(Qg2,cg2,xL,xU,x...) # relaxation of constraint 2

    # Register the user-defined functions
    # Note: if the gradients and Hessians are directly available, they could
    # be passed as arguments to the register function to speed things up.
    JuMP.register(mL,:fcv,2,fcv,autodiff=true)
    JuMP.register(mL,:g1cv,2,g1cv,autodiff=true)
    JuMP.register(mL,:g2cv,2,g2cv,autodiff=true)

    # Declare the objective function and constraints to the JuMP model
    @NLobjective(mL,Min,fcv(x[1],x[2]))
    @NLconstraint(mL,g1cv(x[1],x[2])<=0.)
    @NLconstraint(mL,g2cv(x[1],x[2])<=0.)
    
    # Solve the relaxed problem
    JuMP.optimize!(mL)
    
    # Get primal status, termination status, determine if a global solution was obtained
    tstatus = MOI.get(mL, MOI.TerminationStatus())
    pstatus = MOI.get(mL, MOI.PrimalStatus())

    solution = JuMP.value.(x)
    # Interpret status codes for branch-and-bound
    if EAGO.local_problem_status(tstatus, pstatus) == EAGO.LRS_FEASIBLE
        opt._lower_objective_value = JuMP.objective_value(mL) 
        opt._lower_solution[1:length(solution)] = solution
        opt._lower_feasibility = true
        opt._cut_add_flag = false
    else
        opt._lower_feasibility = false
        opt._lower_objective_value = Inf
        opt._cut_add_flag = false
    end
    return
end;

Caution: By default, EAGO solves the epigraph reformulation of your original problem, which increases the original problem dimensionality by +1 with the introduction of an auxiliary variable. When defining custom routines (such as the lower-bounding problem here) that are intended to work nicely with default EAGO routines (such as preprocessing), the user must account for the *new* dimensionality of the problem. In the code above, we wish to access the information of the specific B&B node and define an optimization problem based on that information. However, in this example, the node has information for 3 variables (the original 2 plus 1 for the auxiliary variable appended to the original variable vector) as $(x_1,x_2,\eta)$. The lower-bounding problem was defined to optimize the relaxed problem with respect to the original 2 decision variables. When storing the results of this subproblem to the current B&B node, it is important to take care to store the information at the appropriate indices and not inadvertently redefine the problem dimensionality (i.e., by simply storing the optimization solution as the $\verb|lower_solution|$ of the current node). For problems that are defined to only branch on a subset of the original variables, the optimizer has a member $\verb|_sol_to_branch_map|$ that carries the mapping between the indices of the original variables to those of the variables being branched on. See the custom_quasiconvex example to see how this is done. 

(Optional) Turn off (short circuit) preprocessing routines if you don't want to use them as defined in EAGO. 

In [ ]:
import EAGO: preprocess!
function EAGO.preprocess!(t::αBB_Convex, x::GlobalOptimizer)
    x._preprocess_feasibility = true
    return
end

(Optional) Turn off (short circuit) postprocessing routines if you don't want to use them as defined in EAGO. 

In [ ]:
import EAGO: postprocess!
function EAGO.postprocess!(t::αBB_Convex, x::GlobalOptimizer)
    x._postprocess_feasibility = true
    return
end

Now, we'll tell EAGO to use our custom/extended solver, set up the main JuMP model, and solve it with our custom solver. 

In [6]:
factory = () -> EAGO.Optimizer(SubSolvers(; t=αBB_Convex() ))
m = JuMP.Model(optimizer_with_attributes(factory,
                                "relative_tolerance" => 1e-4,
                                "verbosity" => 1,
                                "output_iterations" => 1, 
                                "branch_variable" => Bool[true; true],
                                ))
Qf,cf,Qg1,cg1,Qg2,cg2=QCQP_setup()# get QCQP data
xL = [-3.;-5.] # lower bounds on x
xU = [1.; 2] # upper bounds on x
@variable(m, xL[i] <= x[i=1:2] <= xU[i])

#define objective and constraints
@objective(m,Min,1/2*x'*Qf*x+cf'*x)
@constraint(m,1/2*x'*Qg1*x+cg1'*x<=0.)
@constraint(m,1/2*x'*Qg2*x+cg2'*x<=0.)

# solve the problem
@time optimize!(m)

println("x1* = ", JuMP.value(x[1]), " x2* = ",
         JuMP.value(x[2])," f* = ",JuMP.objective_value(m))
TermStatus = JuMP.termination_status(m)
PrimStatus = JuMP.primal_status(m)
println("Algorithm terminated with a status of $TermStatus and result code of $PrimStatus")

-----------------------------------------------------------------------------------------------------------------------------
|  Iteration #  |     Nodes    | Lower Bound  |  Upper Bound  |      Gap     |     Ratio    |     Time     |    Time Left   |
-----------------------------------------------------------------------------------------------------------------------------
|            1  |            2 |   -7.787E+01 |   -8.430E+00 |   6.944E+01 |    8.917E-01 |    2.100E-02 |    3.600E+03 |
|            2  |            3 |   -7.787E+01 |   -8.430E+00 |   6.944E+01 |    8.917E-01 |    4.700E-02 |    3.600E+03 |


|            3  |            4 |   -7.415E+01 |   -5.519E+01 |   1.897E+01 |    2.558E-01 |    6.700E-02 |    3.600E+03 |
|            4  |            3 |   -7.415E+01 |   -5.519E+01 |   1.897E+01 |    2.558E-01 |    9.300E-02 |    3.600E+03 |
|            5  |            2 |   -5.556E+01 |   -5.519E+01 |   3.688E-01 |    6.639E-03 |    1.090E-01 |    3.600E+03 |
|            6  |            3 |   -5.556E+01 |   -5.519E+01 |   3.688E-01 |    6.639E-03 |    1.390E-01 |    3.600E+03 |
|            7  |            2 |   -5.520E+01 |   -5.519E+01 |   1.424E-02 |    2.580E-04 |    1.500E-01 |    3.600E+03 |
|            8  |            3 |   -5.520E+01 |   -5.519E+01 |   1.424E-02 |    2.580E-04 |    1.590E-01 |    3.600E+03 |


|            9  |            2 |   -5.520E+01 |   -5.519E+01 |   1.424E-02 |    2.580E-04 |    1.640E-01 |    3.600E+03 |
|           10  |            1 |   -5.520E+01 |   -5.519E+01 |   1.424E-02 |    2.580E-04 |    1.700E-01 |    3.600E+03 |
|           11  |            2 |   -5.519E+01 |   -5.519E+01 |   2.453E-03 |    4.444E-05 |    1.830E-01 |    3.600E+03 |
 
Relative Tolerance Achieved
First Solution Found at Node 11
LBD = -55.189304808350606
UBD = -55.18685223769234
Solution is:
    X[1] = 0.9999999990283819
    X[2] = -4.236067970190563
 
  0.184824 seconds (128.86 k allocations: 7.445 MiB)
x1* = 0.9999999990283819 x2* = -4.236067970190563 f* = -55.18685223769234
Algorithm terminated with a status of OPTIMAL and result code of FEASIBLE_POINT
